In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import optim, nn, utils
import numpy as np
import layers
import lenet5_cifar10 as models
from torchvision import datasets, transforms
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import time

root = os.getcwd()
ckpt_dir = Path(root) / 'checkpoints'
seed = 2022
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(2022)

c:\ANACONDA\envs\dnerf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform = transforms.ToTensor()
root = "./CIFAR10_DATASET"
train_dataset = datasets.CIFAR10(root, transform=transform, train=True, download=True)
train_dataset, valid_dataset = utils.data.random_split(train_dataset, [40000, 10000])
test_dataset = datasets.CIFAR10(root, transform=transform, train=False, download=True)

print("Train : ", len(train_dataset))
print("Validation : ", len(valid_dataset))
print("Test : ", len(test_dataset))

train_batchsize = 64
test_batchsize = 256

train_dataloader = utils.data.DataLoader(train_dataset, batch_size=train_batchsize, shuffle=True)
valid_dataloader = utils.data.DataLoader(valid_dataset, batch_size=test_batchsize, shuffle=False)
test_dataloader = utils.data.DataLoader(test_dataset, batch_size=test_batchsize, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
Train :  40000
Validation :  10000
Test :  10000


In [3]:
def mnist_experiment(bnn_model, train_dataloader, valid_dataloader, use_aleatoric=True, **kwargs):
  if bnn_model == "gaussian":
    model = models.gaussian_lenet5(kwargs["var_type"], dict(), kwargs["is_lrt"], use_aleatoric)
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    # Every forward gives different output.
    bnn_type = "random"
  elif bnn_model == "dropout":
    model = models.dropout_lenet5(kwargs["dropout_rate"], kwargs["dropout_type"], dict(), use_aleatoric)
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"], weight_decay=(1 - kwargs["dropout_rate"]) / 2 * kwargs["num_sample"])
    # Every forward gives different output.
    bnn_type = "random"
  elif bnn_model == "ensemble":
    model = models.ensemble_lenet5(kwargs["num_ensemble"], use_aleatoric)
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    # Forward gives [batch_size * num_ensemble, output_shape]
    bnn_type = "ensemble"
  elif bnn_model == "swag":
    model = models.lenet5(use_aleatoric)
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    # SWAG is trained as simple NN.
    bnn_type = "swag"
  elif bnn_model == "batchensemble":
    model = models.batchensemble_lenet5(kwargs["num_models"], use_aleatoric)
    optimizer = optim.SGD(model.parameters(), kwargs["lr"], kwargs["momentum"])
    # Forward gives [batch_size * num_models, output_shape]
    bnn_type = "ensemble"
    kwargs["num_ensemble"] = kwargs["num_models"]
  else:
    raise ValueError("No bnn model choosen.")

  criterion = nn.CrossEntropyLoss()
  train_loss_res = []
  valid_loss_res = []
  train_acc_res = []
  valid_acc_res = []

  model = model.to(device)

  # Load the checkpoint
  if False:
    file_name = 'd_model_'+kwargs['dropout_type']+'.pth'
    ckpt_path = ckpt_dir / file_name
    try:
      checkpoint = torch.load(ckpt_path)
      model.load_state_dict(checkpoint)
      train_loss_res = checkpoint['tr_los']
      train_acc_res = checkpoint['tr_ac']
      valid_loss_res = checkpoint['te_los']
      valid_acc_res = checkpoint['te_ac']
      print('>>Load the checkpoint DONE!!')
      return model, train_loss_res, train_acc_res, valid_loss_res, valid_acc_res
    except Exception as e:
      print(e)

  for epoch in range(kwargs["epoch"]):
    start_time = time.time()
    train_loss = 0.0
    train_acc_count = 0
    model.train()
    for data in train_dataloader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      if bnn_type == "random":
        for ind_sample in range(kwargs["num_sample"]):
          outputs = model(images)
          if use_aleatoric:
            output_mean, output_std = torch.chunk(outputs, 2, dim=1)
            eps = torch.normal(0, 1, output_mean.shape, device=device)
            outputs = output_mean + eps * output_std
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          train_loss += loss.item() / kwargs["num_sample"]
          output_pred = torch.argmax(outputs.detach(), dim=1)
          train_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["num_sample"]
          
      elif bnn_type == "ensemble":
        outputs = model(images)
        if use_aleatoric:
          output_mean, output_std = torch.chunk(outputs, 2, dim=1)
          eps = torch.normal(0, 1, output_mean.shape)
          outputs = output_mean + eps * output_std
        labels = labels.repeat(kwargs["num_ensemble"]) # [y1, y2, ..., y1, y2, ..., ] with num_ensemble times
        loss = criterion(outputs, labels) * kwargs["num_ensemble"]
        loss.backward()
        optimizer.step()

        train_loss += loss.item() / kwargs["num_ensemble"]
        output_pred = torch.argmax(outputs.detach(), dim=1)
        train_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["num_ensemble"]
      else:
        outputs = model(images)
        if use_aleatoric:
          output_mean, output_std = torch.chunk(outputs, 2, dim=1)
          eps = torch.normal(0, 1, output_mean.shape)
          outputs = output_mean + eps * output_std
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        output_pred = torch.argmax(outputs.detach(), dim=1)
        train_acc_count += torch.count_nonzero(output_pred == labels).item()
    train_loss_res.append(train_loss)
    train_acc_res.append(train_acc_count / len(train_dataset))
    print(f'training {epoch+1}/{kwargs["epoch"]} || time: {time.time()-start_time: 04f} secs')

    start_time = time.time()
    model.eval()
    valid_loss = 0.0
    valid_acc_count = 0
    with torch.no_grad():
      for data in valid_dataloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        if bnn_type == "random":
          for ind_sample in range(kwargs["valid_num_sample"]):
            outputs = model(images)
            if use_aleatoric:
              output_mean, output_std = torch.chunk(outputs, 2, dim=1)
              eps = torch.normal(0, 1, output_mean.shape, device=device)
              outputs = output_mean + eps * output_std
            loss = criterion(outputs, labels)
            
            valid_loss += loss.item() / kwargs["valid_num_sample"]
            output_pred = torch.argmax(outputs.detach(), dim=1)
            valid_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["valid_num_sample"]
        elif bnn_type == "ensemble":
          outputs = model(images)
          if use_aleatoric:
            output_mean, output_std = torch.chunk(outputs, 2, dim=1)
            eps = torch.normal(0, 1, output_mean.shape)
            outputs = output_mean + eps * output_std
          labels = labels.repeat(kwargs["num_ensemble"]) # [y1, y2, ..., y1, y2, ..., ] with num_ensemble times
          loss = criterion(outputs, labels) * kwargs["num_ensemble"]
          valid_loss += loss.item() / kwargs["num_ensemble"]
          output_pred = torch.argmax(outputs.detach(), dim=1)
          valid_acc_count += torch.count_nonzero(output_pred == labels).item() / kwargs["num_ensemble"]
        else:
          outputs = model(images)
          if use_aleatoric:
            output_mean, output_std = torch.chunk(outputs, 2, dim=1)
            eps = torch.normal(0, 1, output_mean.shape)
            outputs = output_mean + eps * output_std
          loss = criterion(outputs, labels)
          valid_loss += loss.item()
          output_pred = torch.argmax(outputs.detach(), dim=1)
          valid_acc_count += torch.count_nonzero(output_pred == labels).item()
    valid_loss_res.append(valid_loss)
    valid_acc_res.append(valid_acc_count / len(valid_dataset))
    print(f'validation {epoch+1}/{kwargs["epoch"]} || time: {time.time()-start_time: 04f} secs')
  
  return model, train_loss_res, train_acc_res, valid_loss_res, valid_acc_res

In [4]:
res = dict()

In [ ]:
d_model_w, tr_los, tr_ac, te_los, te_ac = mnist_experiment("dropout", train_dataloader, valid_dataloader, dropout_rate=0.2, dropout_type='w', num_sample=1, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["d_w"] = (tr_los, tr_ac, te_los, te_ac)

In [6]:
d_model_f, tr_los, tr_ac, te_los, te_ac = mnist_experiment("dropout", train_dataloader, valid_dataloader, dropout_rate=0.2, dropout_type='f', num_sample=1, valid_num_sample=20, epoch=100, lr=0.00001, momentum=0.1)
res["d_f"] = (tr_los, tr_ac, te_los, te_ac)

training 1/100 || time:  43.738837 secs
validation 1/100 || time:  56.755390 secs
training 2/100 || time:  40.409618 secs
validation 2/100 || time:  54.746855 secs
training 3/100 || time:  38.775697 secs
validation 3/100 || time:  53.563621 secs
training 4/100 || time:  43.883694 secs
validation 4/100 || time:  60.148113 secs
training 5/100 || time:  38.287073 secs
validation 5/100 || time:  55.692036 secs
training 6/100 || time:  38.727453 secs
validation 6/100 || time:  52.882654 secs
training 7/100 || time:  38.423238 secs
validation 7/100 || time:  54.878464 secs
training 8/100 || time:  38.852731 secs
validation 8/100 || time:  54.095202 secs
training 9/100 || time:  38.738855 secs
validation 9/100 || time:  53.742454 secs
training 10/100 || time:  40.136639 secs
validation 10/100 || time:  52.245506 secs
training 11/100 || time:  39.048207 secs
validation 11/100 || time:  56.074375 secs
training 12/100 || time:  39.044566 secs
validation 12/100 || time:  51.663712 secs
training 1

In [ ]:
d_model_c, tr_los, tr_ac, te_los, te_ac = mnist_experiment("dropout", train_dataloader, valid_dataloader, dropout_rate=0.2, dropout_type='c', num_sample=1, valid_num_sample=20, epoch=100, lr=0.00001, momentum=0.1)
res["d_c"] = (tr_los, tr_ac, te_los, te_ac)

In [ ]:
os.makedirs(ckpt_dir, exist_ok=True)

# torch.save({
#     'model' : d_model_w.state_dict(),
#     'tr_los' : tr_los,
#     'tr_ac' : tr_ac,
#     'te_los' : te_los,
#     'te_ac' : te_ac,
# }, ckpt_dir / 'd_model_w.pth')
torch.save({
    'model' : d_model_f.state_dict(),
    'tr_los' : tr_los,
    'tr_ac' : tr_ac,
    'te_los' : te_los,
    'te_ac' : te_ac,
}, ckpt_dir / 'd_model_f.pth')
torch.save({
    'model' : d_model_c.state_dict(),
    'tr_los' : tr_los,
    'tr_ac' : tr_ac,
    'te_los' : te_los,
    'te_ac' : te_ac,
}, ckpt_dir / 'd_model_c.pth')

In [ ]:
g_model_sq, tr_los, tr_ac, te_los, te_ac = mnist_experiment("gaussian", train_dataloader, valid_dataloader, var_type='sq', is_lrt=True, num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["g_sq"] = (tr_los, tr_ac, te_los, te_ac)
model, tr_los, tr_ac, te_los, te_ac = mnist_experiment("gaussian", train_dataloader, valid_dataloader, var_type='exp', is_lrt=True, num_sample=5, valid_num_sample=20, epoch=5, lr=0.00001, momentum=0.1)
res["g_exp"] = (tr_los, tr_ac, te_los, te_ac)

In [ ]:
e_model, tr_los, tr_ac, te_los, te_ac = mnist_experiment("ensemble", train_dataloader, valid_dataloader, num_ensemble = 5, epoch=5, lr=0.00001, momentum=0.1)
res["e"] = (tr_los, tr_ac, te_los, te_ac)

In [ ]:
model, tr_los, tr_ac, te_los, te_ac = mnist_experiment("swag", train_dataloader, valid_dataloader, epoch=5, lr=0.00001, momentum=0.1)
res["s"] = (tr_los, tr_ac, te_los, te_ac)

In [ ]:
b_model, tr_los, tr_ac, te_los, te_ac = mnist_experiment("batchensemble", train_dataloader, valid_dataloader, num_models=5, epoch=5, lr=0.00001, momentum=0.1)

In [ ]:
for k, v in res.items():
  print(k, v[0])

In [1]:
sys.path.append("..")
import matplotlib.cm as cm
from attack import pgd
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# pgd_attack_config = {
#     'eps' : 8.0/255.0, 
#     'attack_steps': 7,
#     'attack_lr': 2.0 / 255.0, 
#     'random_init': False, 
# }
# use_aleatoric = True
# criterion = nn.CrossEntropyLoss()
# num_sample = 5
# model = d_model_w
# pgd = pgd.PGD(model, pgd_attack_config)
# test_loss = 0.
# test_acc_count = 0

my_image = None
for data in test_dataloader:
    images, labels = data
    my_image = images[0]
    if True:
        break
print(my_image.size())
plt.figure(2, figsize=(20,6))
plt.imshow(my_image, cmap=cm.gray)

pgd(x.unsqueeze(dim=0),y.unsqueeze(dim=0))

# for data in test_dataloader:
#   images, labels = data
#   adver_images = []
#   for x,y in zip(images, labels):
#       adver_images.append(pgd(torch.unsqueeze(x,dim=0),torch.unsqueeze(y,dim=0)))
#   adver_images = torch.cat([item for item in adver_images], dim=0)
  
#   for ind_sample in range(num_sample):
#     outputs = model(images)
#     if use_aleatoric:
#         output_mean, output_std = torch.chunk(outputs, 2, dim=1)
#         eps = torch.normal(0, 1, output_mean.shape)
#         outputs = output_mean + eps * output_std
#     loss = criterion(outputs, labels)

#     test_loss += loss.item() /num_sample
#     output_pred = torch.argmax(outputs.detach(), dim=1)
#     test_acc_count += torch.count_nonzero(output_pred == labels).item() / num_sample
  
  

# attack_config = {
#     'eps' : 8.0/255.0,
#     'random_init' : False,
# }

# fgsm = FGSM(model, attack_config)
# adversarial_image = fgsm(image, label)

NameError: name 'torch' is not defined

In [29]:
a = [1,2,3,4]
b = [5,6,7,8]
for i_a,i_b in zip(a,b):
    i_a = i_b
print(a)

[1, 2, 3, 4]


In [42]:
# os.getcwd()
new_path = os.path.join('..', os.getcwd())
print(new_path)

c:\Users\곽상원\Desktop\Bayes_Adversarial\models
